# Decision Trees

**Fall 2024 - Instructor:  Chris Volinsky**


**Original Notebooks courtesy of Prof. Foster Provost and Rubing Li**

***

## Some general imports

In [ ]:
import os
import numpy as np
import pandas as pd
import math
import matplotlib.pylab as plt
import seaborn as sns

%matplotlib inline
sns.set(style='ticks', palette='Set2')

## Predicting who will survive the Titanic

This time we will use a clasic introductory dataset that contains demographic and traveling information for the Titanic passengers. The goal is to predict the survival of these passengers. We will only keep a few variables of interest and transform all of them to numeric variables. We will also drop some outliers.

In [ ]:
# Read in the data
# Download data to your local machine from this URL:
# https://drive.google.com/uc?export=download&id=1mJgS2IOvkmJLrnoVNSNcIz4e07V3wFwA

# upload it into Colab

from google.colab import files
uploaded = files.upload()



Saving titanic.csv to titanic.csv


In [ ]:
# read in data into data frame "df" - only the features we will use

df = pd.read_csv("titanic.csv")[["survived", "pclass", "sex", "age", "fare"]]
df.head()


In [ ]:
#replace "survived" with a categorical feature with values "lived" and "died"

df["survived"] = df["survived"].astype(bool)
df["survived"] = df["survived"].map({1.0: "lived", False: "died"})

# make a barplot of survived

df["survived"].value_counts().plot(kind="bar")
plt.show()

### Exploring the Titanic Data

In [ ]:
# look at descriptives (note: only shows numeric)

df.describe()

It looks like there might be some missing values of some of these variables, becasue they have different counts.  Let's confirm

In [ ]:
na_count = df.isna().sum()

na_count

Wow.  There are a lot of missing values for the age variable.  We would want to check and see if the missing cases are different in any way.  For starters, are they more or less likely to die?  Lets check.


In [ ]:
age_missing = df['age'].isnull()

print("Percent missing age",age_missing.mean().round(4))

survived_missing = df[age_missing]['survived'].value_counts()
survived_non_missing = df[~age_missing]['survived'].value_counts()

n_missing=survived_missing.sum()

print("\nAge Missing that Survived\n", (survived_missing/n_missing).round(3))

n_nonmissing=survived_non_missing.sum()

print("\nAge Non-Missing that Survived\n", (survived_non_missing/n_nonmissing).round(3))


This is concerning.   In a more thorough analysis of this data, we would study the missing-ness more deeply to see if the missing values are correlated with other attributes, which would potentially add bias to the results  Also, we would consider options for filling in - imputing - the data.

However, in order to discuss the things we want to discuss, we will drop all cases with missing values, which is easy to do in python.   We will drop missing values, which helps us to get to the things we want to discuss.

In [ ]:
df = df.dropna()
df
df["female"] = df["sex"]
df["female"] = (df.sex == "female").astype(int)


We'd like to use data about the passengers to predict whether they will survive. Let's start by taking a look at how well some of the variables "split" the data according to our target.

In [ ]:
# what is the *base rate* of survival?


survival = df["survived"].value_counts()/df["survived"].count()
print(survival.round(3))

In [ ]:
# do fares differ from those that survived from those who died?
# how do you interpret this?

sns.boxplot(x="survived", y="fare", width=0.4, data=df)
plt.show()

Above we see boxplots that show the fare distribution grouped by our target variable (survival). Alternatively, let's plot the distribution of fare according to survival:

In [ ]:
for r in ["died","lived"]:
    x_max=300 # maybe shrink to 300 to avoid outliers
    hist = df[df.survived == r].hist('fare',bins=range(0,x_max,10))
    plt.title("survived =" + str(r))
    # plt.ylim(0,300) # can use this to plot on same axes
    plt.xlabel("fare")
    plt.xlim([0,x_max])
    plt.show()

On might conclude from this that people that paid less are less likely to survive. **What do you think is a good split point?**. How effective is this threshold? Let's quantify it!

### Finding the best splits


**Entropy** ($H$) and **information gain** ($IG$) are useful tools for measuring the effectiveness of a split on the values of one variable for giving information on the value of another variable. Entropy measures how random data is, information gain is a measure of the reduction in randomness after performing a split.

We wrote Python functions to calculate entropy and information gain


In [ ]:
def entropy(target_column):
    """
        computes -sum_i p_i * log_2 (p_i) for each i
    """
    # get the counts of each target value
    target_counts = target_column.value_counts().astype(float).values
    total = target_column.count()
    # compute probas
    probas = target_counts/total
    # p_i * log_2 (p_i)
    entropy_components = probas * np.log2(probas)
    # return negative sum
    return - entropy_components.sum()

def information_gain(df, info_column, target_column, threshold):
    """
        computes H(target) - H(target | info > thresh) - H(target | info <= thresh)
    """
    # split data
    data_above_thresh = df[df[info_column] > threshold]
    data_below_thresh = df[df[info_column] <= threshold]
    # get entropy
    H = entropy(df[target_column])
    entropy_above = entropy(data_above_thresh[target_column])
    entropy_below = entropy(data_below_thresh[target_column])
    # compute weighted average
    ct_above = data_above_thresh.shape[0]
    ct_below = data_below_thresh.shape[0]
    tot = float(df.shape[0])
    return H - entropy_above*ct_above/tot - entropy_below*ct_below/tot

Now that we have a way of calculating $H$ and $IG$, let's test our prior guess as a good split point as a split on fare allows us to predict people who will survive.

In [ ]:
threshold = ####
prior_entropy = entropy(df["survived"])
IG = information_gain(df, "fare", "survived", threshold)
print ("IG of %.4f - using a threshold of %.2f - given a prior entropy of %.4f" % (IG, threshold, prior_entropy))

How good was our guess? Let's loop through all possible splits on fare and see what is the best!

In [ ]:
def best_threshold(df, info_column, target_column, criteria=information_gain):
    maximum_ig = 0
    maximum_threshold = 0

    for thresh in df[info_column].unique():
        IG = criteria(df, info_column, target_column, thresh)
        if IG > maximum_ig:
            maximum_ig = IG
            maximum_threshold = thresh

    return (maximum_threshold, maximum_ig)

col = "fare"
maximum_threshold, maximum_ig = best_threshold(df, col, "survived")

print ("The best split threshold for %s is %.2f - which provides a maximum IG of %.4f." % (col, maximum_threshold, maximum_ig ))

Other observed features may also give us (strong) clues about survival.

In [ ]:
# Names of different columns
categorical_cols = ["pclass", "female"]
continuous_cols = ["age", "fare"]
target_col = "survived"
predictor_cols = categorical_cols + continuous_cols

# This is to plot everything in a 2x2 space
rows, cols = 2, 2
fig, axs = plt.subplots(ncols=cols, nrows=rows, figsize=(7*cols, 7*rows))
axs = axs.flatten()
posn = 0

# Plot continous features
for col in continuous_cols:
    sns.boxplot(x=target_col, y=col, data=df, ax=axs[posn])
    axs[posn].set_ylabel(col)
    axs[posn].set_title("")
    posn += 1

# plots for cateogrical variables
pctlived = df.groupby('pclass')['survived'].value_counts(normalize=True)
pctlived = pctlived.unstack()*100
pctlived.index = pctlived.index.astype(str)
pctlived['lived'].plot(kind='bar',ax=axs[posn])
axs[posn].set_ylabel("Percent lived")
axs[posn].set_xlabel("Passenger Class")
axs[posn].set_title("")
posn +=1


pctlived = df.groupby('sex')['survived'].value_counts(normalize=True)
pctlived = pctlived.unstack()*100
pctlived.index = pctlived.index.astype(str)
pctlived['lived'].plot(kind='bar',ax=axs[posn])
axs[posn].set_ylabel("Percent Lived")
axs[posn].set_xlabel("Sex Class")
axs[posn].set_title("")


plt.tight_layout()

So, then ... what feature gives the most effective split?

In [ ]:
def best_split(df, info_columns, target_column, criteria=information_gain):
    maximum_ig = 0
    maximum_threshold = 0
    maximum_column = ""

    for info_column in info_columns:
        thresh, ig = best_threshold(df, info_column, target_column, criteria)

        if ig > maximum_ig:
            maximum_ig = ig
            maximum_threshold = thresh
            maximum_column = info_column

    return maximum_column, maximum_threshold, maximum_ig

max_col, max_threshold, max_ig = best_split(df, predictor_cols, "survived")


print ("The best column to split on is %s giving us a IG of %.4f using a thresh of %.2f" % (max_col, max_ig, max_threshold))

### Using DecisionTreeClassifier

Of course, splitting the data one time sometimes isn't enough to make accurate predictions. However, we can continue to split the data recursively ("recursive partitioning"), building a tree-structured model that may give better results.

Tree-structured models are one of the most popular and powerful sorts of machine learning algorithm.  They include classification or class-probability estimation trees (aka "decision trees"), regression trees, and many more complex models built by combining tree-structured models, such as random forests and gradient boosted models (which usually combine trees).

What are some other ways you might consider splitting the data?

Rather than build a classification tree from scratch (think if you could now do this!) let's use sklearn's implementation which includes some additional functionality.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Define the model (tree)
decision_tree = DecisionTreeClassifier(max_depth=None, criterion="entropy")

# Fit the model
decision_tree.fit(df[predictor_cols], df["survived"])

We now have a classification tree, let's visualize the results!

In [ ]:
from sklearn.tree import plot_tree

plt.figure(figsize=(15,10))
plot_tree(decision_tree, feature_names=predictor_cols, class_names=["died", "survived"])
plt.show()

Whoa!  This will almost surely overfit!  We will see later how to protect against his...

Lets make things easier by setting max_depth to 3 - then visualize the tree and explore it to see what is important

In [ ]:
from sklearn import tree

decision_tree = DecisionTreeClassifier(max_depth=3, criterion="entropy")   # Look at those 2 arguments !!!

# Train the model on our X and y!
decision_tree.fit(df[predictor_cols], df["survived"])

plt.figure(figsize=(15,10))
tree.plot_tree(decision_tree, feature_names=predictor_cols, class_names=["died", "survived"],
               filled=True, impurity=False)
plt.show()


### Evaluating the model

How good is this model?  Calculate the accuracy - although we know this might not be the best metric*, it is still useful!!
It simply means the percent of cases that are correctly predicted in the leaf nodes of the tree.

* because it is not measured on a hold-out set
* also, what is the base-rate?

In [ ]:
from sklearn import metrics
accuracy = metrics.accuracy_score(decision_tree.predict(df[predictor_cols]),df["survived"])
print ( "Accuracy = ", round(accuracy,4))

### Buiding the best tree using training and test sets


Use the sklearn function train_test_split to create a training and test set with an 80/20 split.  Use a random_state argument to make sure you can run with the same split repeatedly

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

### define X and y (as before)

X = df[predictor_cols]
y = df["survived"]

### create training and test sets
### CHOOSE YOUR OWN RANDOM STATE

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=###)

### fit the model on the training set (use depth=2)
depth=2
decision_tree = DecisionTreeClassifier(max_depth=depth, criterion="entropy")
decision_tree.fit(X_train, y_train)

### apply the model to the test set using .predict
y_pred = decision_tree.predict(X_test)

### find the accuracy using metrics.accuracy_score()
accuracy = metrics.accuracy_score(y_pred, y_test)
print("Accuracy:", round(accuracy,4))

Now we will use our training/test split to help us determine what is the best depth for the tree.  There are various parameters we can use to prevent overfitting:

- `max_depth` -  how many layers to the tree
- `min_samples_split` - the minimum number of samples needed at a node to split (default =2)
- `min_samples_leaf` - the minimum number of samples that can be in a leaf node
- `max_leaf_nodes` - limits the number of leaf nodes in the overall tree
- `min_impurity_decrease` - restricts the ability of the tree to create trivial splits

Let's explore the depth of the tree, using `max-depth`

Using a for loop, fit trees to your training set of max_depth = [2,3,4,5,6] and report the accuracy on the test set for each.

What is YOUR optimal depth? (this might differ based on the `random_state` chosen)

Optional - change `random_state` and run again.

In [ ]:
depth_vals = [2,3,4,5,6,7,8]
for i in depth_vals:
  depth = i
  decision_tree = DecisionTreeClassifier(max_depth=depth, criterion="entropy")
  # Tell the model what is the "training" data and then train it
  decision_tree.fit(X_train, y_train)

  y_pred = decision_tree.predict(X_test)

  accuracy = metrics.accuracy_score(y_pred, y_test)

  print("max_depth = ",i,"Accuracy:", round(accuracy,4))

Once we know the optimal depth, if this were a real-world problem, before we apply it to the real world, we would re-fit our model to the FULL data set.  (This is a contrived case because there is no actual real-world application to predicting Titanic deaths 😱 ).  But lets do it anyway.


In [ ]:
# now that we know the best max_depth, to find the best tree...apply it to the ENTIRE data set

opt_depth = ##OPT_DEPTH_VALUE##

tree_final = DecisionTreeClassifier(max_depth=opt_depth, criterion="entropy")
tree_final.fit(X,y)


plt.figure(figsize=(25,20))
tree.plot_tree(tree_final, feature_names=predictor_cols, class_names=["died", "survived"],filled=True, impurity=False)
plt.show()

You can use `.feature_importances_` applied to the tree model to get the importance of each feature.  Find the importances of the four attributes and plot as a barplot.

In [ ]:
# find feature importances and plot

# print(predictor_cols, importances.round(4))

scores = tree_final.feature_importances_
names=predictor_cols


sorted_pairs = sorted(zip(scores, names), reverse=True)
sorted_scores, sorted_names = zip(*sorted_pairs)

# Create the bar plot
plt.figure(figsize=(10, 6))  # Optional: Adjust the figure size as needed
plt.bar(sorted_names, sorted_scores, color='skyblue')  # You can change the color

# Add title and labels
plt.title('Scores by Name')
plt.xlabel('Name')
plt.ylabel('Score')

# Display the plot
plt.xticks(rotation=45)  # Rotate names to prevent overlap
plt.show()

## Tayko:  Another example for you to try

Tayko software example [Shmueli].

Task is to predict whether the person made a Purchase  (0,1) given 24 attributes including the source catalog the customer received (encoded in 15 "source" variables) and other customer attributes.  

In [ ]:
# file is at url = "https://drive.google.com/uc?export=download&id=1wo7x7PmnCJ5-79RZXJSIAa7eS8DdrX-y"

uploaded = files.upload()


# read in using read_csv

df=pd.read_csv("Tayko.csv")
df.describe()


Now do the same exercise here that we did above, splitting 80/20 into train/test.

We can look at a different parameter: min_leaf_vals - which determines the minimum number of observations allowed in a leaf nodes.  This will remove small nodes and protect against overfitting.

Use the test set to determine the best value of `min_leaf_vals`.

In [ ]:
# Build a tree using 80/20 split

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# make sure to remove ["sequence_number","Purchase","Spending"] when creating your X data
# target variable is "Purchase"

X = df.drop(["sequence_number","Purchase","Spending"],axis=1)
y = df["Purchase"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=123)


In [ ]:

# cycle through min_leaf_vals and find the one with the best accuracy on the test data

min_leaf_vals = list(range(2,31,2))

for ml in min_leaf_vals:

  decision_tree = DecisionTreeClassifier(min_samples_leaf=ml, criterion="entropy")

  decision_tree.fit(X_train, y_train)

  y_pred = decision_tree.predict(X_test)

  accuracy = metrics.accuracy_score(y_pred, y_test)

  print("min_leaf= ",ml,"Accuracy:", round(accuracy,3))


In [ ]:
#Fit your final tree on the FULL data set - with the optimized value of min_leaf_vals, and plot it.

best_min_leaf_val = ###

tree_final = DecisionTreeClassifier(min_samples_leaf=best_min_leaf_val, criterion="entropy")

# fit ALL data using your optimized parameter
tree_final.fit(X,y)

X.names = list(X.columns)

plt.figure(figsize=(55,40))
tree.plot_tree(tree_final, feature_names=X.names, class_names=["NoPurchase", "Purchase"],filled=True, impurity=False)
plt.show()

In [ ]:
scores = tree_final.feature_importances_
sorted_pairs = sorted(zip(scores.round(4), X.names), reverse=True)
sorted_pairs


### Do-it-Yourself Class Exercise

For the Tayko data,
- optimize on the parameter `min_impurity_decrease` - use values (0,001, 0.002, ..., 0.010)
- use 5-fold cross-validation to fit the model  
- find the optimal tree using parameter "f1"
